In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from src.HMM import estimate_parameters, get_hmm_forecasts, params_to_vec
from src.backtest import find_optimal_trade, portfolio_value, find_optimal_holdings, port_value

In [2]:
# Load from csv and compute results
msci_data = pd.read_csv('MSCI World Index_11_22_21-12_31_93.csv')
returns = msci_data.set_index('Date').sort_index().pct_change().dropna().query('Date >= "1997-01-01" and Date <= "2015-12-31"')
log_returns = np.log(1 + returns.Close.values)[:,np.newaxis]

In [3]:
N_eff = 260
f = 1 - 1/N_eff
A = 1/N_eff
K = 100
# h = np.array([[0], [100]])
# kappa = np.array([[0], [0]])
# gamma = 0
# rho = np.zeros((2,1))
V = 100
h = 0
kappa = 0.001
gamma = 0
rho = 0

In [ ]:
# theta = None
value = []
holding = [0]
thetas = []
seed = np.ones(K-1)

for i in range(len(log_returns)-N_eff-1):
    l_ret_t = log_returns[i:i+N_eff]
    delta, Gamma, pi = estimate_parameters(log_returns[i:i+N_eff], method="newton", theta_0=theta, memLength=N_eff)
    if np.isnan(delta).any() or np.isnan(Gamma).any():
        delta, Gamma, pi = estimate_parameters(l_ret_t, theta_0=None)    
    forecast_means, forecast_var = get_hmm_forecasts(K, delta, Gamma, pi, l_ret_t)
    theta = params_to_vec(delta, Gamma, pi)
    thetas.append(theta)
    h = find_optimal_holdings(V, h, forecast_means, forecast_var, kappa, gamma, rho, seed=seed)
    holding.append(h[0])
    V = port_value(V, holding[i-1], [np.exp(log_returns[i+N_eff])-1,0], [0], [h[0]], kappa, 0, 0)
    value.append(V)
    seed = h
    print(V)
    if np.isnan(V).any():
        break

101.39728619044857
102.23659738614003
101.84443881576992
103.7256632620382
104.35753001225093
105.32661524326504
105.1560000766732
104.82268200791019
104.94303757937928
105.44061871512882
106.47188765806861
107.20538800196637
107.56143990089575
107.1485534265504
109.1906977294866
109.90748202179142
110.13983513893314
110.68043224744846
111.17433585184756
110.87021515320019
111.53306231903181
111.67570595929146
111.6420509754177
110.78708065661134
110.90175277053885
111.40902921621229
112.14130371632653
111.69587666154695
112.07276790479553
112.65738394917226
111.79115815562665
112.929855590137
113.70581470508077
114.25945033383861
114.99774261252634
115.076530935701
114.04414758932168
112.61225060987134
114.49581530336262
114.69050158424828
115.4529764175738
115.54636342580629
115.61946829143938
116.50931943790299
116.82157528165891
117.44363528160376
117.20281269832164
118.45417632056835
118.5137969045831
119.24272819437877
119.60479787188162
119.56367638496302
119.28697001092806
118.

119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
119.35308498836982
121.47250936232963
120.91415223943113
120.65315611426051
120.97610921433551
121.85705403029252
121.92109518478084
122.80762939144022
123.52856605661206
123.929107478471
124.14339903387423
125.32503412253232
125.36055991089493
126.36835547516532
126.26187284551315
126.2991986071528
125.92110948353903
126.0184975113704
126.6573932892835
127.11288126380992
126.56663674194648
124.6410442772576
126.01688700896462
127.9664475388688
128.38536763524064
128.23625405955332
127.73993511226881
128.52680764064164
128.3331684102065
127.50385440666983
127.61696851681637
128.68918417731766
129.0590313180313
129.40547880614344
129.8580299821652
130.8291629328454
132.

102.82302345751668
102.1111093952346
102.23510008074624
102.47509051777382
103.30274570383254
103.01682089612929
102.80440415499726
102.20356928570432
101.80808105198207
102.39795029460042
102.76782680329912
103.77399860220277
103.32123237674209
102.97202429803559
101.52982602514774
99.54309320500964
100.04938874102073
99.45246170844767
99.06467072887065
100.66464897509759
101.07393921443727
100.44721504529437
100.47605724867303
100.30424979937968
101.08083974356882
100.74745627203056
99.42066944360334
98.44157208484944
99.1447401717138
99.77057098569868
100.28141855939198
100.4356393224143
101.3355662776092
101.90425836094602
102.59557693952134
102.01244149166781
101.18180403409535
101.40893413573566
99.9984457078631
99.49526317826692
99.84140182885892
99.92066980991662
100.35310012676416
100.33215127988251
100.02336018008037
98.66282858948047
98.78498140754083
98.57160215438272
99.20657967467804
98.7159543270751
100.22513907914512
99.97649266536371
98.69837900905634
97.88052041439649

In [ ]:
thetas

In [ ]:
forecast_var

In [ ]:
# theta = None
# value = []
# holding = [0]
# thetas = []
# seed = np.ones(K-1)
# trades = []
# range(len(log_returns)-N_eff-4617)
for i in range(last_i, len(log_returns)-N_eff):
    l_ret_t = log_returns[i:i+N_eff]
    delta, Gamma, pi = estimate_parameters(l_ret_t, theta_0=theta)
    if np.isnan(delta).any() or np.isnan(Gamma).any():
        delta, Gamma, pi = estimate_parameters(l_ret_t, theta_0=None)
    forecast_means, forecast_var = get_hmm_forecasts(K, delta, Gamma, pi, l_ret_t)
    theta = params_to_vec(delta, Gamma, pi)
    thetas.append(theta)
    h = find_optimal_holdings(V, h, forecast_means, forecast_var, kappa, gamma, rho, seed=seed)
    holding.append(h[0])
    V = port_value(V, h[0], [np.exp(log_returns[i+N_eff])-1,0], [0], [h[0]], kappa, 0, 0)
    value.append(V)
    seed = h
    print(V)
    if np.isnan(V).any():
        break

In [ ]:
plt.plot(np.array(value).flatten())

In [ ]:
plt.plot(holding)

In [ ]:
plt.plot(holding)

In [ ]:
delta

In [ ]:
Gamma

In [ ]:
pi

In [ ]:
thetas

In [ ]:
seed.tofile('opt-seed')

In [ ]:
value0 = np.fromfile('strategy_0')

In [ ]:
holding0 = np.fromfile('holding_strategy_0')

In [ ]:
np.array(value).flatten().tofile('strategy_1')

In [ ]:
np.array(holding).flatten().tofile('holding_strategy_1')

In [ ]:
pickle.dump(np.array(thetas),open("params_strategy_1.pkl",'wb'))

In [ ]:
forecast_means

In [ ]:
figure, ax = plt.subplots(1, figsize=(20,10))
ax.plot(dIndex[N_eff:].to_pydatetime(),value0, label=r'MPC($\gamma = 0,\kappa=0.001,\rho=0$) Run 1')
ax.plot(dIndex[N_eff+1:].to_pydatetime(),value, label=r'MPC($\gamma = 0,\kappa=0.001,\rho=0$) Run 2')
ax.plot(dIndex[N_eff:].to_pydatetime(),100*np.exp(log_returns[N_eff:].cumsum()),label='MSCI World')
ax.fill_between(dIndex[N_eff:].to_pydatetime(), 0.5, 1, np.array(holding) == 0,
                color='gray', alpha=0.5, transform=ax.get_xaxis_transform())
ax.fill_between(dIndex[N_eff:].to_pydatetime(), 0, 0.5, np.array(holding0)[1:] == 0,
                color='gray', alpha=0.5, transform=ax.get_xaxis_transform())
ax.set_xlabel('Date',fontdict={'size':16})
ax.set_ylabel('Index',fontdict={'size':16})
ax.set_ylim(70,250)
plt.legend(loc='upper left',fontsize=20)
plt.show()
# plt.savefig('backtest-v1.png')

In [ ]:
dIndex = pd.DatetimeIndex(returns.index.values)

In [ ]:
dIndex[N_eff:].to_pydatetime()